In [3]:
import MVTecAD_loaddata as mv
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import csv
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import cv2
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras import optimizers
from efficientnet.tfkeras import EfficientNetB2
from tensorflow.keras.layers import Dense, UpSampling2D, Conv2D, Conv2DTranspose, Reshape, Input, BatchNormalization, LeakyReLU, Activation, Dropout, Flatten 
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import load_img, img_to_array, array_to_img
# from keras.utils.vis_utils import plot_model
import random 

In [6]:
def OCSVM(X, x_test, y, y_test):
    tuned_parameters = [
        {'nu': [2**-3], 'kernel': ['rbf'], 'gamma': [10**-4, 10**-5, 10**-6]},
    ]
    score = 'roc_auc_ovr_weighted'
    clf = GridSearchCV(
        OneClassSVM(),
        tuned_parameters,
        cv=5,
        scoring=score)
    clf.fit(X, y)
    a = clf.decision_function(x_test)
    b = clf.predict(x_test)
    r1 = roc_auc_score(y_test,a)
    r2 = f1_score(y_test,b)
    r3 = accuracy_score(y_test,b)
    print('roc_auc : ',r1)
    print('f1_score : ', r2)
    print('accuracy_score : ', r3)
    print('best_params : ', clf.best_params_)
    result = [r1,r2,r3]
#     return result

def model_predict(X_train, X_test):
    input_shape = (256, 256, 3)
    inputs = Input(input_shape)
    base_model = EfficientNetB2(include_top=False, weights='imagenet')(inputs)
    model = Model(inputs=inputs, outputs=base_model)
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss="mean_squared_error")
    f_tr = model.predict(X_train, verbose=0)
    f_ts = model.predict(X_test, verbose=0)
    x_tr = np.reshape(f_tr, (f_tr.shape[0], f_tr.shape[1] * f_tr.shape[2] * f_tr.shape[3]))
    x_ts = np.reshape(f_ts, (f_ts.shape[0], f_ts.shape[1] * f_ts.shape[2] * f_ts.shape[3]))
    return x_tr, x_ts

def OCSVM_SV(X, y, train_data):
    tuned_parameters = [
    {'nu': [2**-3, 2**-4, 2**-5, 2**-6], 'kernel': ['rbf'], 'gamma': [10**-3, 10**-4, 10**-5, 10**-6, 10**-7]},
    ]
    score = 'roc_auc_ovr_weighted'
    clf = GridSearchCV(
        OneClassSVM(),
        tuned_parameters,
        cv=5,
        scoring=score)
    clf.fit(X, y)
    ocsvm = OneClassSVM(nu=clf.best_params_['nu'], gamma=clf.best_params_['gamma'])
    ocsvm.fit(X,y)
    sv_num = ocsvm.support_
    sv = train_data[sv_num,:]
    print('best_params : ', clf.best_params_)
    return clf, ocsvm, sv

In [40]:
def train_gan_data(X_train):
    # ハイパーパラメータの設定
    # 潜在変数の次元数
    latent_dim = 1000
    # バッチサイズ
    batch_size = 1024
    # エポック数
    epochs = 1
    
    # 学習率
    lr = 0.001
    # Generatorモデルの構築
    def build_generator():
        # Generatorは、潜在変数からRGB画像を生成するモデルです。
        # Conv2DTranspose層とBatchNormalization層とLeakyReLU層を用いて、アップサンプリングを行います。
        model = Sequential()
        # 入力層
        model.add(Dense(8*8*512, input_dim=latent_dim))
        model.add(Reshape((8, 8, 512)))
        # 中間層1
        model.add(Conv2DTranspose(256, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層2
        model.add(Conv2DTranspose(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層3
        model.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層4
        model.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 出力層
        model.add(Conv2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='tanh'))
        return model

    # Discriminatorモデルの構築
    def build_discriminator():
        # Discriminatorは、RGB画像が本物か偽物かを判定するモデルです。
        # Conv2D層とBatchNormalization層とLeakyReLU層を用いて、ダウンサンプリングを行います。
        model = Sequential()
        # 入力層
        model.add(Conv2D(64, kernel_size=5, strides=2, padding='same', input_shape=(256, 256 ,3)))
        model.add(LeakyReLU(alpha=0.2))
        # 中間層1
        model.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層2
        model.add(Conv2D(256, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層3
        model.add(Conv2D(512, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 出力層
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        return model

    # Generatorモデルの作成
    generator = build_generator()
    # Discriminatorモデルの作成
    discriminator = build_discriminator()
    # Discriminatorモデルのコンパイル
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr), metrics=['accuracy'])
    # Discriminatorモデルの学習を停止
    discriminator.trainable = False

    # GANモデルの構築
    # GANモデルは、GeneratorモデルとDiscriminatorモデルを結合したモデルです。
    # 潜在変数を入力として、Discriminatorモデルの出力を得るようにします。
    gan_input = Input(shape=(latent_dim,))
    gan_output = discriminator(generator(gan_input))
    gan = Model(gan_input, gan_output)
    # GANモデルのコンパイル
    gan.compile(loss='binary_crossentropy', optimizer=Adam(lr))

    # 学習のループ
    for epoch in range(epochs):
        # バッチごとに学習
        for i in range(0, len(X_train), batch_size):
            # 本物の画像データをバッチサイズ分取得
            real_images = X_train[i:i+batch_size]
            # 偽物の画像データをバッチサイズ分生成
            latent_vectors = np.random.normal(0, 1, size=(len(real_images), latent_dim))
            fake_images = generator.predict(latent_vectors)
            # 本物の画像データと偽物の画像データを結合
            images = np.concatenate([real_images, fake_images])
            # 本物の画像データにはラベル1を、偽物の画像データにはラベル0を付ける
            labels = np.concatenate([np.ones((len(real_images), 1)), np.zeros((len(real_images), 1))])
            # ラベルにわずかなノイズを加える
#             labels += 0.05 * np.random.random(labels.shape)
            # Discriminatorモデルの学習を再開
            discriminator.trainable = True
            # Discriminatorモデルの学習と評価
            loss, acc = discriminator.train_on_batch(images, labels)
            # Discriminatorモデルの学習を停止
            discriminator.trainable = False
            # 潜在変数をバッチサイズ分生成
            latent_vectors = np.random.normal(0, 1, size=(len(real_images), latent_dim))
            # 偽物の画像データにはラベル1を付ける（騙す）
            misleading_labels = np.ones((len(real_images), 1))
            # GANモデルの学習
            gan.train_on_batch(latent_vectors, misleading_labels)
        # エポックごとに結果を表示
        print(f'Epoch {epoch}, Loss: {loss}, Accuracy: {acc}')
    imageslatent_vectors = np.random.normal(0, 1, size=(len(X_train), latent_dim))
    fake_images = generator.predict(imageslatent_vectors)
    fake_images = fake_images/np.max(fake_images)*255
    return fake_images, generator
def sv_mask(X):
    
    X_list = []
    for i in range(len(X)):
        for j in range(2):
            rand_x = random.randint(32,224)
            rand_y = random.randint(32,224)
            x1, x2 = rand_x-20, rand_x+20
            y1, y2 = rand_y-20, rand_y+20
            X[i,x1:x2, y1:y2, :] = 1.0
            X_list.append(X[i])
    X_list = np.array(X_list)*255
    return X_list

def save_svmask(img, name):
    for i in range(len(img)):
        cv2.imwrite('sv_fakes/'+name+'/'+str(i)+'.png', img[i])

def make_dataset(fake_image, train_data, y_train):
    fake_image = fake_image[:int(len(train_data)*0.125)]/255
    x_train = np.concatenate([train_data, fake_image])
    y = []
    for i in range(len(fake_image)):
        y.append(-1)
    y = np.array(y)
    y = y.reshape([len(y), 1])
    labels = np.concatenate([y_train, y])
    return x_train, labels

In [41]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# f_train, f_test = model_predict(X_train, X_test)
# clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
# SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'bottle')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

7/7 [==============================] - 9s 1s/step
Epoch 0, Loss: 0.3972645401954651, Accuracy: 0.9174311757087708
7/7 [==============================] - 9s 1s/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.9920634920634921
f1_score :  0.8
accuracy_score :  0.9156626506024096
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


NameError: name 'res' is not defined